In [7]:
# 03_preprocess_feature_engineering.ipynb
"""
Notebook 03 — Pré‑processamento & Feature Engineering
Transforma o Chicago Crime bruto num dataset pronto para ML.
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, dayofweek, to_timestamp, col
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler, Imputer
)
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("Crime_FE").getOrCreate()

INPUT_PATH  = "../dados/chicago_crime.parquet"
OUTPUT_DATA = "../dados/chicago_ready.parquet"
OUTPUT_PIPE = "../dados/pipeline_fe"

# ────────────────────────────────────────────────────────────────────────────────
# 1. Ler dados brutos
# ────────────────────────────────────────────────────────────────────────────────

df = spark.read.parquet(INPUT_PATH)

# ────────────────────────────────────────────────────────────────────────────────
# 2. Correções de timestamp e features temporais
# ────────────────────────────────────────────────────────────────────────────────

# Formato real: "03/18/2015 12:00:00 AM"  ➜ pattern "MM/dd/yyyy hh:mm:ss a"
df = df.withColumn("Date", to_timestamp("Date", "MM/dd/yyyy hh:mm:ss a"))

# Descarta registos sem data válida (muito poucos)
df = df.filter(col("Date").isNotNull())

# Colunas derivadas
df = (
    df.withColumn("Hour", hour("Date"))        # 0‑23
      .withColumn("DayOfWeek", dayofweek("Date")) # 1‑7 (Dom=1)
)

# ────────────────────────────────────────────────────────────────────────────────
# 3. Definir colunas para FE
# ────────────────────────────────────────────────────────────────────────────────

cat_cols  = ["Primary Type", "Location Description"]
num_cols  = ["Beat", "District", "Latitude", "Longitude", "Hour", "DayOfWeek"]
label_col = "Arrest"   # ainda booleano; convertemos depois para int

# ────────────────────────────────────────────────────────────────────────────────
# 4. Pipeline de Feature Engineering
# ────────────────────────────────────────────────────────────────────────────────

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
            for c in cat_cols]
encoders = [OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_vec")
            for c in cat_cols]

# Imputação para colunas numéricas sujeitas a nulos
num_imp_cols = ["Beat", "District", "Latitude", "Longitude"]

imputer = Imputer(inputCols=num_imp_cols,
                  outputCols=[f"{c}_imp" for c in num_imp_cols])

assembler = VectorAssembler(
    inputCols=[f"{c}_vec" for c in cat_cols] +
              [f"{c}_imp" for c in num_imp_cols] + ["Hour", "DayOfWeek"],
    outputCol="features",
    handleInvalid="keep"  # evita falha se restar null
)

pipeline_fe = Pipeline(stages=indexers + encoders + [imputer, assembler])

# ────────────────────────────────────────────────────────────────────────────────
# 5. Fit & Transform
# ────────────────────────────────────────────────────────────────────────────────

model_fe = pipeline_fe.fit(df)

# Converte rótulo boolean → inteiro (1/0) e guarda como "label"
df_ready = (
    model_fe.transform(df)
            .withColumn("label", col(label_col).cast("int"))
            .select("features", "label")
)

# ────────────────────────────────────────────────────────────────────────────────
# 6. Persistir dataset e pipeline
# ────────────────────────────────────────────────────────────────────────────────

df_ready.write.mode("overwrite").parquet(OUTPUT_DATA)
model_fe.write().overwrite().save(OUTPUT_PIPE)

print("✅ Pré‑processamento concluído – dados em", OUTPUT_DATA)

spark.stop()


✅ Pré‑processamento concluído – dados em ../dados/chicago_ready.parquet
